# import Libraries

In [304]:
import numpy as np
import pandas as pd

from sqlalchemy import create_engine
from sqlalchemy import URL
from sqlalchemy import text
from urllib.parse import quote_plus

# create sql URL

In [305]:
url_object = URL.create(
    "mysql+mysqlconnector",
    username="root",
    password="",
    host="localhost"
    )

# create sengine

In [306]:
engin = create_engine(url_object)

# Fetch the required DATA

In [307]:
query = ('''SELECT `club_full_name`, `club_id`,sum(PPG) as PPG, sum(goals) as goals,sum(own_goals) as own_goals,sum(yellow_cards) as yellow_cards,sum(second_yellow_cards) as second_yellow_cards,sum(red_cards) as red_cards,sum(clean_sheets) as clean_sheets, sum(goals_conceded) as goals_conceded, sum(assists) as assists,sum(minutes_per_goal) as minutes_per_goal FROM `player_statistics` 
WHERE season =2021 and appearances !=0 and competition in ('Premier League', 'Bundesliga', 'Serie A', 'Ligue 1', 'LaLiga') 
GROUP BY club_full_name;''') 

In [308]:
with engin.connect() as conn:
    conn.execute(text("USE quera_project1"))
    df = pd.read_sql(text(query), conn)
    df = df.fillna(0)

In [309]:
df.head(200)

,club_full_name,club_id,PPG,goals,own_goals,yellow_cards,second_yellow_cards,red_cards,clean_sheets,goals_conceded,assists,minutes_per_goal
0,1.FC Köln,3.0,38.76,52.0,2.0,66.0,1.0,0.0,4.0,49.0,45.0,4904.0
1,1.FC Union Berlin,89.0,44.48,50.0,4.0,61.0,1.0,0.0,8.0,44.0,38.0,5453.0
2,1.FSV Mainz 05,39.0,42.29,49.0,1.0,58.0,2.0,2.0,10.0,45.0,40.0,6175.0
3,AC Milan,5.0,64.59,66.0,2.0,74.0,1.0,2.0,18.0,31.0,45.0,4816.0
4,ACF Fiorentina,430.0,44.87,59.0,2.0,78.0,5.0,3.0,12.0,51.0,42.0,6762.0
...,...,...,...,...,...,...,...,...,...,...,...,...
95,VfL Wolfsburg,82.0,44.45,42.0,1.0,58.0,1.0,2.0,11.0,54.0,33.0,5558.0
96,Villarreal CF,0.0,47.10,62.0,1.0,77.0,1.0,0.0,14.0,37.0,53.0,4835.0
97,Watford FC,0.0,19.23,33.0,3.0,57.0,3.0,0.0,4.0,77.0,25.0,2024.0
98,West Ham United,379.0,33.09,58.0,2.0,48.0,2.0,1.0,8.0,51.0,49.0,5009.0


# Normalize columns


In [310]:


import pandas as pd
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()

numeric_columns = df.select_dtypes(include=np.number).columns.tolist()
df_normalized = pd.DataFrame(scaler.fit_transform(df[numeric_columns]), columns=numeric_columns)

df_final = pd.merge(df['club_full_name'], df_normalized, left_index=True, right_index=True)

df_final.head()

,club_full_name,club_id,PPG,goals,own_goals,yellow_cards,second_yellow_cards,red_cards,clean_sheets,goals_conceded,assists,minutes_per_goal
0,1.FC Köln,0.000126,0.438032,0.536842,0.333333,0.547009,0.166667,0.000000,0.190476,0.538462,0.556962,0.679213
1,1.FC Union Berlin,0.003735,0.505676,0.515789,0.666667,0.504274,0.166667,0.000000,0.380952,0.483516,0.468354,0.757485
2,1.FSV Mainz 05,0.001637,0.479778,0.505263,0.166667,0.478632,0.333333,0.285714,0.476190,0.494505,0.493671,0.860422
3,AC Milan,0.000210,0.743496,0.684211,0.333333,0.615385,0.166667,0.285714,0.857143,0.340659,0.556962,0.666667
4,ACF Fiorentina,0.018048,0.510289,0.610526,0.333333,0.649573,0.833333,0.428571,0.571429,0.560440,0.518987,0.944112


In [311]:
pos_attack_metrics = [ "goals", "assists", ]
pos_attack_weight = [70, 30 ]

neg_attack_metrics = ["minutes_per_goal"]
neg_attack_weight = [20]

pos_defence_metrics = ["clean_sheets" ]
pos_defence_weight = [20 ]
neg_defence_metrics = ["own_goals", "yellow_cards", "second_yellow_cards", "red_cards", "goals_conceded"]
neg_defence_weight = [10, 10, 10, 30, 40]


def attack_metric(row):
    row['attack_metric'] = sum([pos_attack_weight[i] * row[pos_attack_metrics[i]] for i in range(len(pos_attack_metrics))]) - sum([neg_attack_weight[i] * row[neg_attack_metrics[i]] for i in range(len(neg_attack_metrics))])

    total_weight = sum(pos_attack_weight) + sum(neg_attack_weight)
    row['attack_metric'] = row['attack_metric']/total_weight
    return row['attack_metric']

def defence_metric(row):
    row['defence_metric'] = sum([pos_defence_weight[i] * row[pos_defence_metrics[i]] for i in range(len(pos_defence_metrics))]) - sum([neg_defence_weight[i] * row[neg_defence_metrics[i]] for i in range(len(neg_defence_metrics))])
    total_weight = sum(pos_defence_weight) + sum(neg_defence_weight)
    row['defence_metric'] = row['defence_metric']/total_weight
    return row['defence_metric']



In [312]:
attack_metric(df_final)
defence_metric(df_final)
df_final.head(5)


,club_full_name,club_id,PPG,goals,own_goals,yellow_cards,second_yellow_cards,red_cards,clean_sheets,goals_conceded,assists,minutes_per_goal,attack_metric,defence_metric
0,1.FC Köln,0.000126,0.438032,0.536842,0.333333,0.547009,0.166667,0.000000,0.190476,0.538462,0.556962,0.679213,0.339196,-0.234992
1,1.FC Union Berlin,0.003735,0.505676,0.515789,0.666667,0.504274,0.166667,0.000000,0.380952,0.483516,0.468354,0.757485,0.291718,-0.209147
2,1.FSV Mainz 05,0.001637,0.479778,0.505263,0.166667,0.478632,0.333333,0.285714,0.476190,0.494505,0.493671,0.860422,0.274751,-0.238451
3,AC Milan,0.000210,0.743496,0.684211,0.333333,0.615385,0.166667,0.285714,0.857143,0.340659,0.556962,0.666667,0.427252,-0.135073
4,ACF Fiorentina,0.018048,0.510289,0.610526,0.333333,0.649573,0.833333,0.428571,0.571429,0.560440,0.518987,0.944112,0.328535,-0.350071


In [313]:
# df_final[['club_full_name', 'attack_metric', 'defence_metric']].sort_values(by=['attack_metric', 'defence_metric'],
                            #  ascending=[False, True])
df_final['diff'] =  df_final['attack_metric'] -  df_final['defence_metric']

df_final[['club_full_name', 'diff']].sort_values(by=['diff'],
                             ascending=[False])


,club_full_name,diff
38,FC Girondins Bordeaux,0.884166
75,SS Lazio,0.872440
89,US Sassuolo,0.846676
92,Venezia FC,0.839281
16,Bayern Munich,0.838078
...,...,...
29,Cádiz CF,0.365584
72,Real Sociedad,0.358254
99,Wolverhampton Wanderers,0.298559
74,SK Sturm Graz,0.075486


In [314]:
!jupytext --to py -o PlayerRequest.py PlayerRequest.ipynb

[jupytext] Reading PlayerRequest.ipynb in format ipynb
[jupytext] Writing PlayerRequest.py (destination file replaced)
